# Get Information of a Linux Kernel Module

## 1. `modinfo` -- Show a specific module infomation

`modinfo` extracts information from the Linux Kernel modules given on the command line. If the module name is not a filename, then the _/lib/modules/&lt;kernel version&gt; directory_ is searched, as is also done by `modprobe` when loading kernel modules.

In [1]:
modinfo --help

Usage:
	modinfo [options] filename [args]
Options:
	-a, --author                Print only 'author'
	-d, --description           Print only 'description'
	-l, --license               Print only 'license'
	-p, --parameters            Print only 'parm'
	-n, --filename              Print only 'filename'
	-0, --null                  Use \0 instead of \n
	-F, --field=FIELD           Print only provided FIELD
	-k, --set-version=VERSION   Use VERSION instead of `uname -r`
	-b, --basedir=DIR           Use DIR as filesystem root for /lib/modules
	-V, --version               Show version
	-h, --help                  Show this help


Let's reuse an previous _Hello World_, [Linux Kernel Module Parameters](module_parameters.ipynb), to demonstrate how to get information from a linux kernel module.

In [2]:
make

make -C /lib/modules/6.1.29/build M=/home/clymber/Studio/kerdevel/module_param modules
make[1]: Entering directory '/home/clymber/Studio/linux-6.1.29'
make[1]: Leaving directory '/home/clymber/Studio/linux-6.1.29'


### 1.1 `modinfo <filename>`

This is the most straightforward usage of `modinfo` command, in which the _filename_ is the path to the module file. This will print all the information found in the _filename_:

In [3]:
modinfo "module_param.ko"

filename:       /home/clymber/Studio/kerdevel/module_param/module_param.ko
description:    A Hello, World Module
author:         Clymber Loong
license:        GPL
srcversion:     77F5712DDBB750D3644A994
depends:        
retpoline:      Y
name:           module_param
vermagic:       6.1.29 SMP preempt mod_unload modversions 
parm:           irq:The interrupt line number. (int)
parm:           debug:Debug mode, 1 is on, 0 is off. (int)
parm:           name:Your name (charp)


### 1.2 `modinfo <module_name>`

The *module_name* here is not a file name, but a module name, and in this case it would be *module_param*:

In [4]:
modinfo module_param

modinfo: ERROR: Module module_param not found.


: 1

Opps, sorry, there's an ERROR showed up. In the case of `modinfo <module_name>`, `modinfo` command looks for a module whos file name is *module_name.ko* within directry */lib/modules/&lt;kernel version&gt;*. But the *module_param.ko* has not been installed yet. Let's install the module first, and the try it again:

In [5]:
sudo make modules_install

make -C /lib/modules/6.1.29/build M=/home/clymber/Studio/kerdevel/module_param modules_install
make[1]: Entering directory '/home/clymber/Studio/linux-6.1.29'
  INSTALL /lib/modules/6.1.29/extra/module_param.ko
  SIGN    /lib/modules/6.1.29/extra/module_param.ko
  DEPMOD  /lib/modules/6.1.29
make[1]: Leaving directory '/home/clymber/Studio/linux-6.1.29'


In [6]:
modinfo module_param

filename:       /lib/modules/6.1.29/extra/module_param.ko
description:    A Hello, World Module
author:         Clymber Loong
license:        GPL
srcversion:     77F5712DDBB750D3644A994
depends:        
retpoline:      Y
name:           module_param
vermagic:       6.1.29 SMP preempt mod_unload modversions 
sig_id:         PKCS#7
signer:         Build time autogenerated kernel key
sig_key:        52:A1:4F:D3:79:3E:E8:33:24:7A:36:91:3F:A3:49:88:74:59:D2:6F
sig_hashalgo:   sha512
signature:      D0:B1:64:C8:89:80:56:DC:D5:37:B8:77:1C:00:1D:70:E2:D5:E0:FC:
		F3:43:09:D9:93:D1:27:F0:76:2F:CB:D4:71:0D:4C:29:3F:56:57:13:
		5A:51:CC:75:FB:8F:7C:37:08:1F:F6:79:B3:0F:3B:94:6E:63:2F:D1:
		A3:AB:91:07:84:E6:3C:D6:33:77:08:28:D0:81:CB:12:B4:2C:2F:FF:
		81:1A:33:C4:5C:01:76:FD:FC:FF:D9:DB:7E:BF:6A:E1:5C:AD:0B:5E:
		CB:6C:83:A2:FF:1D:47:47:1F:B8:41:4B:3E:0F:2D:11:69:E4:CA:C6:
		28:D4:AC:94:C7:BD:27:51:9B:47:91:BA:D8:40:E9:F6:8B:9C:58:26:
		87:12:13:45:8B:A1:74:D8:7F:58:70:54:88:D8:E4:00:9C:BC:29:D6:

Bingo, we made it! The output shows that, the module file name is */lib/modules/$(uname -r)/extra/module_param.ko*, just as what we expected.

### 1.3 `modinfo -F|--field <fieldname>` - Print for Specific Field only

This is very useful for shell script.  
For example, the following command will print the parameters that a module supports:

In [7]:
modinfo --field "parm" module_param.ko

irq:The interrupt line number. (int)
debug:Debug mode, 1 is on, 0 is off. (int)
name:Your name (charp)


## 2. `lsmod` and */proc/modules* -- Show the status of modules in the Linux Kernel

`lsmod` is a trivial program which nicely formats the contents of the */proc/modules*, showing what kernel modules are currently loaded.

In [8]:
lsmod | head

Module                  Size  Used by
ipheth                 16384  0
apple_mfi_fastcharge    20480  0
rfcomm                 86016  4
xt_conntrack           16384  1
nft_chain_nat          16384  3
xt_MASQUERADE          20480  1
nf_nat                 49152  2 nft_chain_nat,xt_MASQUERADE
nf_conntrack_netlink    49152  0
nf_conntrack          167936  4 xt_conntrack,nf_nat,nf_conntrack_netlink,xt_MASQUERADE


In [ ]:
head /proc/modules

In [9]:
sudo dmesg -C
sudo insmod module_param.ko; sudo dmesg -c
lsmod | grep 'module_param'

[ 6370.486366] Hello, World! name=Hello World, irq=10, debug=0
module_param           16384  0


In [10]:
sudo rmmod module_param.ko; sudo dmesg -c
lsmod | grep 'module_param' || echo "module not found"

[ 6372.782770] Bye Bye, World!
module not found


## Clean Up

In [11]:
sudo rm /lib/modules/$(uname -r)/extra/module_param.ko
sudo depmod -a
make clean

make -C /lib/modules/6.1.29/build M=/home/clymber/Studio/kerdevel/module_param clean
make[1]: Entering directory '/home/clymber/Studio/linux-6.1.29'
  CLEAN   /home/clymber/Studio/kerdevel/module_param/Module.symvers
make[1]: Leaving directory '/home/clymber/Studio/linux-6.1.29'
